In [ ]:
import os

import numpy as np
import pandas as pd
from resolve_path import *

### Leitura dos dados

In [ ]:
# se não existir o folder 'predicao', ele cria
path = "data/util/"

path_relativo = ajuste_path(path)
if not os.path.exists(path_relativo + 'predicao'):
    os.makedirs(path_relativo + 'predicao')

# BACALHAU pois o arquivo que deve ser lido é um csv
df_saida = pd.read_excel(
    path_relativo + 'predicao/LR_dataset-saida-com-probabilidade.xlsx', engine='calamine')

# carrega dataset de local coordenada
df_local_coordenada = pd.read_csv(
    path_relativo + 'local_coordenada.csv', sep=';')

### Pré-processamento do df_local_coordenada

Seleciona apenas o vão do meio de cada linha de transmissão 

In [ ]:
# Se o ultimo caracter for R e o primeiro S-L remove o R
df_local_coordenada['Local de instalação'] = df_local_coordenada['Local de instalação'].apply(
    lambda x: x[:-1] if x[-1] == 'R' and x[:2] == 'S-L' else x)

### Renomeia colunas do dataset de saida

In [ ]:
# BACALHAU Corrige nome da coluna do df_saida
df_saida.rename(
    columns={'Probabilidade de Acidente': 'Probabilidade', 'Local de Instalação': 'Local de instalação'}, inplace=True)

# df_saida.columns

# df_local_coordenada.columns

### Merge do dataframe de saída com o local_coordenadas  

In [ ]:
# Merge dos dataframes por Local de Instalação
df_mapa = pd.merge(df_saida[['Ano', 'Mes', 'Local de instalação', 'Quantidade de Acidentes',
                   'Quantidade de Acidentes de Alto Potencial', 'Probabilidade', 'HH por mes']], df_local_coordenada, on='Local de instalação', how='left')

### Remove Linhas de transmissão do dataset do mapa

In [ ]:
# Remove as linhas onde 'Local de instalação' começa com 'S-L'
print(df_mapa.shape)
df_mapa = df_mapa.loc[~df_mapa['Local de instalação'].str.startswith('S-L')]

print(df_mapa.shape)

### Verifica o separador adequado para exportar o dataframe do mapa

In [ ]:
# Função para verificar se um separador está presente nos dados
def check_separator(df, separator):
    for col in df.columns:
        if df[col].astype(str).str.contains(separator).any():
            return False
    return True


# Lista de separadores comuns para verificar
separators = [',', ';', '|', '\t', '@', '#', '$', '%', '&', '*']

# Encontra um separador adequado
chosen_separator = None
for sep in separators:
    print("Testando com separador:", sep)
    if check_separator(df_mapa, sep):
        chosen_separator = sep
        print("Separador encontrado:", sep)
        break

### Exporta dataset mapa para csv 

In [ ]:
df_mapa.to_csv(path_relativo + 'dataset_mapa.csv', sep=',', index=False)